資料庫建立(可自行至sql server中建立)


CREATE TABLE weather_data (


    id INT IDENTITY(1,1) PRIMARY KEY,    
    region VARCHAR(50),
    startDate = Date,
    endDate = Date,
    max_temperature FLOAT,
    min_temperature FLOAT
);

資料庫連線資訊及使用

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, text, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# SQL Server connection string
server = 'LAPTOP-O5VTIAH6'
database = 'weather'
username = 'sa'
password = 'pai5384'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'


# 定義資料庫表結構
Base = declarative_base()

class WeatherData(Base):
    __tablename__ = 'weather_data'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    region = Column(String(50))
    startDate = Column(Date)
    endDate = Column(Date)
    max_temperature = Column(Float)
    min_temperature = Column(Float)

def Select(date):
    query = text('SELECT * FROM weather_data WHERE startDate = \'' + date + '\'')

    results = session.execute(query)
    return results

def check(date,region):
    query = text('SELECT count(*) FROM weather_data WHERE startDate = \'' + date + '\' AND region = \'' + region +'\'')

    results = session.execute(query).fetchone()

    return results

def update(date,region, max, min):
    query = text('UPDATE weather_data SET max_temperature = \''+max+'\',min_temperature = \''+min+'\' WHERE startDate = \'' + date + '\' AND region = \'' + region +'\'')

    session.execute(query)


爬蟲

In [ ]:
import requests
from dateutil.parser import parse
from datetime import datetime



def getData():
    url = 'https://opendata.cwa.gov.tw/fileapi/v1/opendataapi/F-C0032-001?Authorization=CWA-0003FCAC-184A-4015-B266-BD07DEC47EA5&downloadType=WEB&format=JSON'
    data = requests.get(url)   # 取得 JSON 檔案的內容為文字
    data_json = data.json()    # 轉換成 JSON 格式
    # 取出 location 的內容
    location = data_json['cwaopendata']['dataset']['location']

    # 將資料存入DataBase
    for i in location:
        record = WeatherData(
            region=i['locationName'],
            max_temperature=i['weatherElement'][1]['time'][0]['parameter']['parameterName'],
            min_temperature=i['weatherElement'][2]['time'][0]['parameter']['parameterName'],
            startDate=parse((i['weatherElement'][2]['time'][0]['startTime'])),
            endDate=parse(i['weatherElement'][2]['time'][0]['endTime'])
        )
        
        # 在存進db時，會確認是否有同地區同時間的資料，若有就進行更新舊資料，否則就加入新資料
        if (check(i['weatherElement'][2]['time'][0]['startTime'], record.region)[0] > 0):
            update(i['weatherElement'][2]['time'][0]['startTime'],
                   record.region, record.max_temperature, record.min_temperature)
            print('')
        else:
            session.add(record)

    session.commit()

資料整理

In [ ]:
import pandas as pd

def dataTrans(data):
    regions = []
    highs = []
    lows = []
    for row in data:
        regions.append(row.region)
        highs.append(row.max_temperature)
        lows.append(row.min_temperature)

    df = pd.DataFrame(
        {'region': regions, 'Max Temperature': highs, 'Min Temperature': lows})
    
    return df

繪製分析圖表

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
def makeFig(df):   
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['region'], y=df['Max Temperature'],
                  mode='lines+markers', name='Max Temperature', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df['region'], y=df['Min Temperature'],
                  mode='lines+markers', name='Min Temperature', line=dict(color='blue')))

    # 設置圖表標題和軸標籤
    fig.update_layout(title={
        'text': 'Daily Max and Min Temperatures',
        'x': 0.5,  # 將標題置中
        'xanchor': 'center'
    },
        xaxis_title='region',
        yaxis_title='Temperature (°C)',
        xaxis=dict(tickangle=45),
    )

    # 將圖表轉為HTML格式
    plot_div = fig.to_html(
        full_html=False)
    
    return plot_div

網頁主程式

In [5]:
from flask import Flask, render_template

app = Flask(__name__)

# 資料庫連線引擎
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()

@app.route('/')
def index():
    
    # 拿資料 ==> 爬蟲
    getData()

    today_date = datetime.today().date()
    
    # 拿取本日資料和整理資料和繪製圖表的動作
    data = Select(today_date.strftime('%Y-%m-%d')) # 拿取本日資料
    df = dataTrans(data) # 整理資料
    session.close()
    
    return render_template('index.html', plot_html=makeFig(df), data=df.to_dict(orient='records'))


if __name__ == '__main__':
    app.run()